# Import Data

Tops code needs to be run before affinity

In [1]:
import pandas as pd
ip_attack_frame = pd.read_csv('./../Tops/cc_attack_frame.csv', index_col=0)
if 'Unlabelled' in ip_attack_frame:
    ip_attack_frame.drop('Unlabelled', axis=1, inplace=True)

In [2]:
a = ip_attack_frame
if 'Totals' in a:
    a.drop('Totals', axis=1,inplace=True)

a['Totals'] = a.transpose().sum()
a = a[a['Totals']>0]

ip_attack_frame = a.transpose()


In [3]:
ip_attack_frame

,BD,BE,BF,BG,BA,BB,BM,BN,BO,JP,...,AT,AW,IN,TZ,AZ,IE,ID,MY,QA,MZ
Malware,631,3335,1,10538,2905,7,19,4,2681,7031,...,7897,2,17166,41,2354,6063,7643,10318,96,22
Phishing,73,382,0,543,27,0,0,0,16,662,...,485,0,1346,4,7,323,1066,1416,0,4
Exploits,3,69,0,141,396,0,0,0,1,79,...,309,0,395,0,2228,52,23,184,0,0
Fraudlent Services,15,1064,0,532,1712,0,0,0,0,391,...,1110,0,2876,0,452,844,239,1232,2,1
Unwanted Programs,0,95,0,24,0,0,0,0,0,629,...,155,0,138,0,0,32,27,168,4,0
Spammers,0,0,0,5,16,0,0,0,0,7,...,34,0,13,0,0,1,4,28,0,0
Totals,722,4945,1,11783,5056,7,19,4,2698,8799,...,9990,2,21934,45,5041,7315,9002,13346,102,27


# Entropy Functions

In [4]:
# Compute the entropy for all attacks ip_attack_frame
import math
def shannon_entropy(arr):
    total = sum(arr)
    Pn = [float(c)/total for c in arr]    
    e_arr = []
    for P in Pn:
        if P == 0.0:
            e_arr.append(0)
        else:
            e_arr.append(P*math.log(P,2))
                
    shannon_e = sum(e_arr)
    
    return -1*shannon_e

def entropy(arr):
    total = sum(arr)
    Pn = [float(c)/total for c in arr]    
    e_arr = []
    for P in Pn:
        if P == 0.0:
            e_arr.append(0)
        else:
            e_arr.append(math.log(P,2))
                
    shannon_e = sum(e_arr)
    
    return -1*shannon_e



## Compute Entropies

In [5]:
# We do not use the pandas frame for the time being.
attack_entropies = {}
for index, row in ip_attack_frame.iterrows():
    attack_entropies[index] = (shannon_entropy(list(row)), entropy(list(row)))

    
attack_entropies

{'Exploits': (3.307357728628694, 1316.0970188033327),
 'Fraudlent Services': (2.5270378469823775, 1480.4138918442125),
 'Malware': (1.0337640001390198, 3352.899107045651),
 'Phishing': (1.7560971932287361, 2010.3746590238995),
 'Spammers': (3.253850642245464, 549.4957602699106),
 'Totals': (1.1313652156871288, 3360.288515375573),
 'Unwanted Programs': (0.5401208191535773, 1107.2418584009574)}

In [6]:
index

'Totals'

In [7]:
import pandas as pd
unique_ips = (ip_attack_frame > 0).astype(int).sum(axis=1)
# Normalising the Shannon Entropies
# we divide the entropy amount by the maximum entropy log(n,2)

norm_attack_entropies = {}

for types in attack_entropies:
    n = unique_ips[types]
    norm_attack_entropies[types] = attack_entropies[types][0]/math.log(n,2)

    
# Output the results
temp = pd.DataFrame.from_dict(attack_entropies, orient='index')
temp.columns = ['Shannon', 'Regular']
print temp
    
# Output the results
temp = pd.DataFrame.from_dict(norm_attack_entropies, orient='index')
temp.columns = ['Normalised Shannon']
print temp


a = temp.transpose()
a[['Totals','Unwanted Programs','Fraudlent Services',
                      'Malware', 'Exploits', 'Phishing', 'Spammers']]





                     Shannon      Regular
Spammers            3.253851   549.495760
Malware             1.033764  3352.899107
Exploits            3.307358  1316.097019
Phishing            1.756097  2010.374659
Totals              1.131365  3360.288515
Fraudlent Services  2.527038  1480.413892
Unwanted Programs   0.540121  1107.241858
                    Normalised Shannon
Spammers                      0.550857
Malware                       0.135759
Exploits                      0.485851
Phishing                      0.246680
Totals                        0.148291
Fraudlent Services            0.367161
Unwanted Programs             0.085195


,Totals,Unwanted Programs,Fraudlent Services,Malware,Exploits,Phishing,Spammers
Normalised Shannon,0.148291,0.085195,0.367161,0.135759,0.485851,0.24668,0.550857
